In [1]:
import argparse
from tflearn.data_utils import to_categorical, pad_sequences
import numpy as np
import pdb
from auxiliares import *
from models import *

      
    #Experimento 1 Cross-Validation
tweets,_ = select_tweets('waseem',None)
    
vocab = gen_vocab(tweets)
    
MAX_SEQUENCE_LENGTH = max_len(tweets)

/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorflow/pyth

Instructions for updating:
non-resource variables are not supported in the long term


/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rikka/opt/anaconda3/lib/python3.7/site-pac

cantidad de usuarios en dataset
1550


In [4]:
TOKENIZER = 'glove'
GLOVE_MODEL_FILE = 'glove.txt'
EMBEDDING_DIM = 200
OPTIMIZER = 'adam'
LEARN_EMBEDDINGS = True
EPOCHS = 10
BATCH_SIZE = 128
SCALE_LOSS_FUN = None
NO_OF_FOLDS = 10
SEED = 42
np.random.seed(SEED)

In [6]:
#Step 1: Training the embeddings with all data
a, p, r, f1 = 0., 0., 0., 0.
a1, p1, r1, f11 = 0., 0., 0., 0.
pn, rn, fn = 0., 0., 0.

X, y = gen_sequence(tweets, vocab, 'binary' )
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
X, y = sklearn.utils.shuffle(X, y)
y = np.array(y)
    
y_train = y.reshape((len(y), 1))
X_temp = np.hstack((X, y_train))
   
model = lstm_model(MAX_SEQUENCE_LENGTH, EMBEDDING_DIM,vocab)
    
    #initializing with random embeddings
shuffle_weights(model)
    
for epoch in range(EPOCHS):
    for X_batch in batch_gen(X_temp, BATCH_SIZE):
        x = X_batch[:, :MAX_SEQUENCE_LENGTH]
        y_temp = X_batch[:, MAX_SEQUENCE_LENGTH]
        try:
            y_temp = to_categorical(y_temp, nb_classes=3)
        except Exception as e:
            print (e)
            #print(y_temp)
#            print(to_categorical(y_temp,nb_classes=3))
        loss, acc = model.train_on_batch(x, y_temp, class_weight=None)


tweets,_ = select_tweets('waseem', None)

    #Extracting learned embeddings 
wordEmb = model.layers[0].get_weights()[0]

    #Step 2: Cross- Validation Using the XGB classifier and the learned embeddings
word2vec_model = create_model(wordEmb,vocab)

tweets = select_tweets_whose_embedding_exists(tweets, word2vec_model)

X, y = gen_data(tweets, word2vec_model,'binary')

cv_object = StratifiedKFold(n_splits=NO_OF_FOLDS, shuffle=True, random_state=42)

Model variation is LSTM
variables
200
27
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 27, 200)           2350200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 27, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 603       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total params: 2,671,603
Trainable params: 2,671,603
Non-trainable params: 0
___

In [ ]:
for train_index, test_index in cv_object.split(X,y):
    X_train, y_train = X[train_index],y[train_index]
    X_test, y_test = X[test_index],y[test_index]
    X_train, y_train = sklearn.utils.shuffle(X_train, y_train)

    X_test, y_test = sklearn.utils.shuffle(X_test, y_test)
        #print(y_train)
    import xgboost as xgb
    clf = xgb.XGBClassifier(use_label_encoder=False)
    clf.fit(X_train, y_train)
    model=clf


In [ ]:

        precision, recall, f1_score,precisionw, recallw, f1_scorew,precisionm, recallm, f1_scorem =evaluate_model(model, X_test, y_test, 'binary')
        
        p += precisionw
        p1 += precisionm
        r += recallw
        r1 += recallm
        f1 += f1_scorew
        f11 += f1_scorem
        pn += precision
        rn += recall
        fn += f1_score
    print_scores(p, p1, r,r1, f1, f11,pn, rn, fn,NO_OF_FOLDS)